# Semana 1: Ciclorrutas de Cali
## Descarga, limpieza y procesamiento de capas vectoriales
Código para visualizar el estado de cumplimiento de las ciclorrutas propuestas en 2015 para la ciudad de Cali, Valle del Cauca.

Para el reto de la semana 1, me propuse descargar los datos, limpiarlos y procesarlos desde código, y no desde QGIS, donde tradicionalmente lo hago. El código descarga desde los sitios del IDESC Cali, los carga a Jupyter, son procesados y limpiados para poder ser transformados.

In [1]:
import geopandas as gpd
import requests, zipfile, io, os, tempfile
import pandas as pd
from shapely.geometry import LineString, MultiLineString
import numpy as np
import pyproj
import warnings

import matplotlib.pyplot as plt

# Descarga de datos

In [2]:
#Función para descarga temporal
def read_temp_shapefile(response):

    # Verificar que la respuesta sea válida
    if response.status_code != 200:
        raise ValueError(f"Error HTTP {response.status_code}: No se pudo descargar el recurso.")
    
    # Crear carpeta temporal
    with tempfile.TemporaryDirectory() as tmpdir:
        # Descomprimir contenido del ZIP
        zipfile.ZipFile(io.BytesIO(response.content)).extractall(tmpdir)
        
        # Buscar shapefile dentro de la carpeta
        shp_path = None
        for root, dirs, files in os.walk(tmpdir):
            for file in files:
                if file.endswith(".shp"):
                    shp_path = os.path.join(root, file)
                    break
            if shp_path:
                break

        if not shp_path:
            raise FileNotFoundError("No se encontró ningún archivo .shp dentro del ZIP descargado.")

        gdf = gpd.read_file(shp_path)      

    return gdf

In [3]:
#Importar Ciclorutas 2015

# 1. Definir URL del servicio
url = "https://ws-idesc.cali.gov.co/geoserver/pot_2014/ows"
params = {
    "service": "WFS",
    "version": "1.0.0",
    "request": "GetFeature",
    "typeName": "pot_2014:mov_jerarquizacion_ciclorutas",
    "outputFormat": "SHAPE-ZIP"
}

# 2. Hacer la solicitud HTTP
response = requests.get(url, params=params)

# 3. Descomprimir en memoria
gdf_2015=read_temp_shapefile(response)

ValueError: Error HTTP 502: No se pudo descargar el recurso.

In [4]:
#Importar Ciclorutas 2025

# 1. Definir URL del servicio
url = "https://ws-idesc.cali.gov.co/geoserver/ows"
params = {
    "service": "WFS",
    "version": "1.0.0",
    "request": "GetFeature",
    "typeName": "movilidad:mt_tt_red_cicloinfraestructura",
    "outputFormat": "SHAPE-ZIP"
}

response = requests.get(url, params=params)

# 3. Descomprimir en memoria
gdf_hoy=read_temp_shapefile(response)

ValueError: Error HTTP 502: No se pudo descargar el recurso.

In [22]:
#RESPALDO: subida local
gdf_hoy = gpd.read_file(r"C:\Users\Alejandro\Downloads\Cosas GIS\Colombia\Cali\mt_tt_red_cicloinfraestructura\mt_tt_red_cicloinfraestructura.shp")  
gdf_2015 = gpd.read_file(r"C:\Users\Alejandro\Downloads\Cosas GIS\Colombia\Cali\mov_jerarquizacion_ciclorutas\mov_jerarquizacion_ciclorutas.shp")  

In [23]:
# Convertir CRS (de EPSG:6249 a WGS84)
gdf_hoy = gdf_hoy.to_crs("EPSG:32618")
gdf_2015 = gdf_2015.to_crs("EPSG:32618")

gdf_2015["id_15"] = gdf_2015.index + 1

# Tratamiento de datos

## Inicial

In [24]:
gdf_2015_existentes=gdf_2015[gdf_2015['condicion']=='Existente']
gdf_2015_propuesta=gdf_2015[gdf_2015['condicion']=='Propuesta']

gdf_2025 = gdf_hoy

# Limpieza  básica
def clean_gdf(gdf):
    gdf = gdf[gdf.geometry.notnull()].copy()
    # mantener solo line geometries preferiblemente
    gdf = gdf[gdf.geometry.type.isin(["LineString", "MultiLineString"])].copy()
    gdf["length_m"] = gdf.geometry.length
    return gdf

gdf_2015_existentes = clean_gdf(gdf_2015_existentes)
gdf_2015_propuesta = clean_gdf(gdf_2015_propuesta)
gdf_2025 = clean_gdf(gdf_2025)

buffer_dist=100

# Buffers para matching
buf_2015_existentes = gdf_2015_existentes.copy()
buf_2015_existentes["geometry"] = buf_2015_existentes.geometry.buffer(buffer_dist, cap_style='flat')

buf_2015_propuesta = gdf_2015_propuesta.copy()
buf_2015_propuesta["geometry"] = buf_2015_propuesta.geometry.buffer(buffer_dist, cap_style='flat')

# buf_2025 = gdf_2025.copy()
# buf_2025["geometry"] = buf_2025.geometry.buffer(buffer_dist, cap_style='flat')

# 1. Explotar la capa (ej. de MultiLineString a LineString)
#    ignore_index=True recalcula el ID (el índice) automáticamente
gdf_2025_exploded = gdf_2025.explode(ignore_index=True)
gdf_2025_exploded["id_25_e"] = gdf_2025_exploded.index + 1

In [28]:
# Comprueba cuántos tipos de geometría hay en tu GDF original
print(gdf_2025.geom_type.value_counts())

LineString    202
Name: count, dtype: int64


In [ ]:
# 2. Crear la copia para el buffer a partir de la capa explotada
buf_2025 = gdf_2025_exploded.copy()

# 3. Aplicar el buffer a la capa ya explotada
buf_2025["geometry"] = buf_2025.geometry.buffer(buffer_dist, cap_style='flat')

## Propuestas pendientes

In [58]:
# 1. Ciclovías propuestas en 2015 que siguen sin cumplirse

# Primero, calcular intersección geométrica real
inter = gpd.overlay(buf_2015_propuesta, gdf_2025, how="intersection", keep_geom_type=False)
print(f"Intersecciones encontradas: {len(inter)}")

inter = inter[inter.geometry.type.isin(["LineString", "MultiLineString"])].copy()

# Calcular longitud de cada intersección
inter["len_inter"] = inter.geometry.length

inter_len = inter.groupby("id_15")["len_inter"].sum().reset_index()

g15_prop_merge = gdf_2015_propuesta.merge(inter_len, on="id_15", how="left")

g15_prop_merge["len_inter"] = g15_prop_merge["len_inter"].fillna(0)

g15_prop_merge["ratio_cumplido"] = g15_prop_merge["len_inter"] / g15_prop_merge["length_m"]

mask_cumplida = g15_prop_merge["ratio_cumplido"] > .75
g15_prop_merge["categoria"] = np.where(mask_cumplida, "0", "propuesta_pendiente")

Intersecciones encontradas: 628


In [59]:
g15_prop_merge.explore(
    column="categoria",  # <-- Este es el parámetro clave
    tiles="CartoDB positron",
    legend=True)

## Ciclovías adicionales

In [62]:
# 2. Ciclovías existentes en 2025 que no fueron propuestas ni existían en 2015

# Primero, calcular intersección geométrica real
inter = gpd.overlay(buf_2025, gdf_2015, how="intersection", keep_geom_type=False)
print(f"Intersecciones encontradas: {len(inter)}")

inter = inter[inter.geometry.type.isin(["LineString", "MultiLineString"])].copy()

# Calcular longitud de cada intersección
inter["len_inter"] = inter.geometry.length

inter_len = inter.groupby("objectid")["len_inter"].sum().reset_index()

gdf_2025_adic = gdf_2025.merge(inter_len, on='objectid', how="left")

gdf_2025_adic["len_inter"] = gdf_2025_adic["len_inter"].fillna(0)

gdf_2025_adic["ratio_cumplido"] = gdf_2025_adic["len_inter"] / gdf_2025_adic["length_m"]

mask_cumplida = gdf_2025_adic["ratio_cumplido"] > .75
gdf_2025_adic["categoria"] = np.where(mask_cumplida, "0", "cv_adicional")

Intersecciones encontradas: 646


In [63]:
gdf_2025_adic.explore(
    column="categoria",  # <-- Este es el parámetro clave
    tiles="CartoDB positron",
    legend=True)

## propuestas cumplidas

In [75]:
# 3. Ciclovías existentes en 2025 que fueron propuestas en 2015

# Primero, calcular intersección geométrica real
inter = gpd.overlay(buf_2025, gdf_2015_propuesta, how="intersection", keep_geom_type=False)
print(f"Intersecciones encontradas: {len(inter)}")

inter = inter[inter.geometry.type.isin(["LineString", "MultiLineString"])].copy()

# Calcular longitud de cada intersección
inter["len_inter"] = inter.geometry.length

inter_len = inter.groupby("objectid")["len_inter"].sum().reset_index()

gdf_2025_cump = gdf_2025.merge(inter_len, on='objectid', how="left")

gdf_2025_cump["len_inter"] = gdf_2025_cump["len_inter"].fillna(0)

gdf_2025_cump["ratio_cumplido"] = gdf_2025_cump["len_inter"] / gdf_2025_cump["length_m"]

mask_cumplida = gdf_2025_cump["ratio_cumplido"] > .75
gdf_2025_cump["categoria"] = np.where(mask_cumplida, "cv_cumplidas", "0")

Intersecciones encontradas: 589


In [76]:
gdf_2025_cump.explore(
    column="categoria",  
    tiles="CartoDB positron",
    legend=True)

## Desde 2015

In [52]:
# 4. Ciclovías existentes en 2025 que existen desde 2015

# Primero, calcular intersección geométrica real
inter = gpd.overlay(buf_2025, gdf_2015_existentes, how="intersection", keep_geom_type=False)
print(f"Intersecciones encontradas: {len(inter)}")

inter = inter[inter.geometry.type.isin(["LineString", "MultiLineString"])].copy()

# Calcular longitud de cada intersección
inter["len_inter"] = inter.geometry.length

inter_len = inter.groupby("objectid")["len_inter"].sum().reset_index()

gdf_2025_exist = gdf_2025.merge(inter_len, on='objectid', how="left")

gdf_2025_exist["len_inter"] = gdf_2025_exist["len_inter"].fillna(0)

gdf_2025_exist["ratio_cumplido"] = gdf_2025_exist["len_inter"] / gdf_2025_exist["length_m"]

mask_cumplida = gdf_2025_exist["ratio_cumplido"] > .75
gdf_2025_exist["categoria"] = np.where(mask_cumplida, "desde_2015", "0")

Intersecciones encontradas: 57


In [53]:
gdf_2025_exist.explore(
    column="categoria",  # <-- Este es el parámetro clave
    tiles="CartoDB positron",
    legend=True)

# Integración

In [ ]:
# Filtrar sólo categorías válidas
g15_prop_merge_f = g15_prop_merge[g15_prop_merge["categoria"].notna() & (g15_prop_merge["categoria"] != "0")].copy()
gdf_2025_adic_f = gdf_2025_adic[gdf_2025_adic["categoria"].notna() & (gdf_2025_adic["categoria"] != "0")].copy()
gdf_2025_cump_f = gdf_2025_cump[gdf_2025_cump["categoria"].notna() & (gdf_2025_cump["categoria"] != "0")].copy()
gdf_2025_exist_f = gdf_2025_exist[gdf_2025_exist["categoria"].notna() & (gdf_2025_exist["categoria"] != "0")].copy()

# Revisar duplicados en IDs dentro de los tres gdf_2025
merged_2025 = pd.concat([gdf_2025_adic_f, gdf_2025_cump_f, gdf_2025_exist_f], ignore_index=True)
duplicados = merged_2025[merged_2025.duplicated(subset="objectid", keep=False)].sort_values("objectid")

if len(duplicados) > 0:
    print(f"Se encontraron {len(duplicados)}  duplicados")
    print("IDs:", duplicados["objectid"].unique().tolist())

In [79]:
# Unificar
unified = gpd.GeoDataFrame(
    pd.concat([g15_prop_merge_f, merged_2025], ignore_index=True),
    crs=gdf_2025.crs
)

print("Integración completada:")
print(unified["categoria"].value_counts())

Integración completada:
categoria
cv_cumplidas           124
cv_adicional            66
desde_2015              14
propuesta_pendiente      5
Name: count, dtype: int64


In [80]:
# Guardar el resultado final
unified.to_file(r"C:\Users\Alejandro\Documents\GitHub\urban-data-workshop\Data\processed\SEM1_ciclorrutas_2015_2025.geojson", driver="GeoJSON")

# Comprobaciones

In [69]:
# 1. Crea el mapa base con el primer GeoDataFrame y guárdalo en 'm'
m = merged_2025.explore(
    color="blue",
    tiles="CartoDB positron",
    name="Propuesta 2015"  # Nombre para el control de capas
)

# 2. Añade el segundo GeoDataFrame (ej. gdf_2025) al mapa 'm'
m = gdf_2025.explore(
    m=m,  # <--- ¡Esta es la clave!
    color="red",
    name="Datos 2025" # Nombre para el control de capas
)

# 3. (Opcional pero recomendado) Añade un control de capas
# Esto te permitirá activar y desactivar las capas en el mapa interactivo.
import folium
folium.LayerControl().add_to(m)

m